# Estimating sample quantiles

# Document

<table align="left">
    <tr>
        <th class="text-align:left">Title</th>
        <td class="text-align:left">Estimating sample quantiles</td>
    </tr>
    <tr>
        <th class="text-align:left">Last modified</th>
        <td class="text-align:left">2019-01-09</td>
    </tr>
    <tr>
        <th class="text-align:left">Author</th>
        <td class="text-align:left">Gilles Pilon <gillespilon13@gmail.com></td>
    </tr>
    <tr>
        <th class="text-align:left">Status</th>
        <td class="text-align:left">Active</td>
    </tr>
    <tr>
        <th class="text-align:left">Type</th>
        <td class="text-align:left">Jupyter notebook</td>
    </tr>
    <tr>
        <th class="text-align:left">Created</th>
        <td class="text-align:left">2018-08-18</td>
    </tr>
    <tr>
        <th class="text-align:left">File name</th>
        <td class="text-align:left">estimating_sample_quantiles.ipynb</td>
    </tr>
    <tr>
        <th class="text-align:left">Other files required</th>
        <td class="text-align:left">estimating_sample_quantiles.csv</td>
    </tr>
</table>

# In brevi

The purpose of this notebook is to explore the ways that Python calculates quantiles. During the development of the anova_one_factor notebook, I discovered that [Python](https://www.python.org), [LibreOffice](https://www.libreoffice.org), and [Excel](https://office.microsoft.com/excel/) calculate quartines in the same way, but that Minitab and an online source calculate them differently. I've discovered that there are at least eleven ways to calculate quantiles.

In addition, this notebook explores how to replicate Minitab's "graphical summary" report.

# Data

Download the data file.

[estimating_sample_quantiles](https://drive.google.com/open?id=1EuMKLYCh2aLllTQoYSGzXNQZKeexaWTn)

# Methodology

Various data munging operations are performed using pandas.

# Explanation of the eleven methods

Quantiles divide the range of a probability distribution into continuous intervals with equal probabilities, or divide the observations in a sample in the same way [Wikipedia](https://en.wikipedia.org/wiki/Quantile). A sample drawn from an unknown population requires estimating the quantiles. There are twelve known methods that commonly appear in statistical packages. Methods 1-3 are based on rounding. Methods 4-9 are based on linear interpolation.

The data are sorted in increasing order. Each method computes $Q_{\text{p}}$, the estimate for the $k^{th}$ $q$-quantile, where $p = k/q$, from a sample of size $N$ by computing a real-valued index $h$. When $h$ is an integer, the $h^{th}$ smallest of the $N$ values, $x_h$, is the quantile estimate. Otherwise, a rounding or interpolation scheme is used to compute the quantile estimate from $h$, $x_{\lfloor \text{h}\rfloor}$, and $x_{\lceil \text{h}\rceil}$.

Sample quantiles are defined by:

$$
\begin{equation}
Q_p = (1 - \gamma) \space x_j \space + \space \gamma \space x_{\text{j+1}}
\end{equation}
$$

where:

$$
\begin{equation}
x_j \text{ is the } j^{th} \text{ order statistic} \\
\gamma \text{ is a function of } j = \lfloor \text{(n} \times \text{p + m)} \rfloor \\
m = \text{alphap + p } \times \text{(1} - \text{alphap} - \text{betap)} \\
g = \text{n} \times \text{p} + \text{m} - \text{j} \\
\frac{j - m}{n} \leq p \lt \frac{j - m + 1}{n}
\end{equation}
$$

Reinterpreting the above equations to compare to <b>R</b> gives:

$$
\begin{equation}
p(k) = \frac{k - \text{alphap}}{n + 1 - \text{alphap} - \text{betap}}
\end{equation}$$

In this notebook I am particularly interested in the 4-quantiles, called the 1<sup>st</sup>, 2<sup>nd</sup> and 3<sup>rd</sup> quartiles (Q1, Q2. Q3).

The following table is a work in progress, to combine the eleven methods into one table.

<table>
<tr>
<th style="text-align:center">Method</th>
<th style="text-align:center">Software</th>
<th style="text-align:center">$h$</th>
<th style="text-align:center">$Q_p$</th>
<th style="text-align:center">SciPy (alphab, betap)</th>
<th style="text-align:center">Notes</th>
</tr>
<tr>
<td style="text-align:center">1</td>
<td>R-1, SAS-3</td>
<td>$Np + \frac{1}{2}$</td>
<td>$x_{\lceil \text{h} - \frac{1}{2}\rceil}$</td>
<td></td>
<td>Inverse of empirical cumulative distribution function (CDF). When $p = 0$, use $x_1$.</td>
</tr>
<tr>
<td style="text-align:center">2</td>
<td>R-2, SAS-5</td>
<td>$Np + \frac{1}{2}$</td>
<td>$\frac{x_{\lceil \text{h} - \frac{1}{2}\rceil} + x_{\lfloor \text{h} + \frac{1}{2}\rfloor}}{2}$</td>
<td></td>
<td>The same as R-1, but with averaging at discontinuities. When $p = 0$, use $x_1$.When $p = 1$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">3</td>
<td>R-3, SAS-2</td>
<td>$Np$</td>
<td>$x_{\lfloor \text{h}\rceil}$</td>
<td></td>
<td>The observation numbered closest to $Np$ (piecewise linear function). It is also called the nearest even-order statistic. Here, $\lfloor \text{h}\rceil$ indicates rounding to the nearest integer, choosing the even integer in the case of a tie. When $p \leq \frac{\frac{1}{2}}{N}$, use $x_1$.</td>
</tr>
<tr>
<td style="text-align:center">4</td>
<td>R-4, SAS-1</td>
<td>$Np$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>(0,1)</td>
<td>$\frac{k}{n}$ Linear interpolation of the emperical distribution function. When $p \lt \frac{1}{N}$, use $x_1$. When $p = 1$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">5</td>
<td>R-5</td>
<td>$Np + \frac{1}{2}$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>(0.5,0.5)</td>
<td>$\frac{k - \frac{1}{2}}{n}$ Piecewise linear function where the knots are the values midway through the steps of the emperical distribution function. When $p \lt \frac{\frac{1}{2}}{N}$, use $x_1$. When $p \geq \frac{(N - \frac{1}{2}}{N})$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">6</td>
<td>R-6, SAS-4 Minitab, SPSS</td>
<td>$(N + 1)p$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>(0,0)</td>
<td>$\frac{k}{n +1}$ Linear interpolation of the expectations for the order statistics for the uniform distribution on [0,1]. That is, it is the linear interpolation between points ($p_h, x_h$), where $p_h = \frac{h}{N+ 1}$ is the probability that the last of $(N + 1)$ randomly drawn values will not exceed the $h^\text{th}$ smallest of the first $N$ randomly drawn values. When $p \lt \frac{1}{N + 1}$, use $x_1$. When $p \geq \frac{N}{N + 1}$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">7</td>
<td>R-7</td>
<td>$(N - 1)p + 1$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>(1,1)</td>
<td>$p(k) = \frac{k - 1}{n - 1}\\ p(k) = \text{mode[F(x[k])]}$<br />Linear interpolation of the modes for the order statistics for the uniform distribution on [0,1].  When $p = 1$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">8</td>
<td>R-8</td>
<td>$(N + \frac{1}{3})p + \frac{1}{3}$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>$(\frac{1}{3},\frac{1}{3})$</td>
<td>$p(k) = \frac{k - \frac{1}{3}}{n + \frac{1}{3}}$ Then $p(k) \text{ ~ median}{[F(x[k])]}.$ The resulting quantile estimtes are approximately median-unbiased regardless of the distribution of x. Linear interpolation of the approximate medians for order statistics. When $p \lt \frac{\frac{2}{3}}{N + \frac{1}{3}}$, use $x_1$. When $p \geq \frac{(N - \frac{1}{3})}{(N + \frac{1}{3})}$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">9</td>
<td>R-9</td>
<td>$(N + \frac{1}{4})p + \frac{3}{8}$</td>
<td>$x_{\lfloor \text{h}\rfloor} + (h - \lfloor \text{h} \rfloor) (x_{\lfloor \text{h} \rfloor + 1} - x_{\lfloor \text{h} \rfloor})$</td>
<td>($\frac{3}{8},\frac{3}{8}$)</td>
<td>$p(k) = \frac{k - \frac{3}{8}}{n + \frac{1}{4}}$ Blom. The resulting quantile estimates are approximately unbiased for the expected order statistics if $x$ is normally distributed. When $p \lt \frac{\frac{5}{8}}{N + \frac{1}{4}}$, use $x_1$. When $p \geq \frac{N - \frac{3}{8}}{N + \frac{1}{4}}$, use $x_N$.</td>
</tr>
<tr>
<td style="text-align:center">10</td>
<td></td>
<td></td>
<td></td>
<td>(0.4,0.4)</td>
<td>Cunnane's approximately quantile unbiased definition.</td>
</tr>
<tr>
<td style="text-align:center">11</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>Filliben's estimate.</td>
</tr>
<tr>
<td style="text-align:center">12</td>
<td></td>
<td></td>
<td></td>
<td>(0.35,0.35)</td>
<td>APL, used with PWM.</td>
</tr>
</table>

# Software

As shown above, R (version 2.0.0 onwards) implements methods 1-9. SciPy implements methods 4-9, 10, and a method called APL (need info on this).

# Does it matter?

TBD

# Code

In [1]:
# Start of time estimation for the notebook.
import datetime as dt
start_time = dt.datetime.now()

In [2]:
# Import the required librairies.
import pandas as pd
import scipy.stats.mstats as ms
from scipy.stats import anderson
import statsmodels.stats.diagnostic as smd

# This is a kluge until I learn to create a package for /datasense
#import sys
#sys.path.insert(0, '/home/gilles/documents/websites/support.gillespilon.ca/fil/datasense')

import datasense as ds

In [3]:
# Read the data file.
# y is the column of response values.
df = pd.read_csv('estimating_sample_quantiles.csv')

In [4]:
# Calculate basic statistics.
df.describe()

,y
count,8.000000
mean,20.875000
std,27.010249
min,0.000000
25%,0.750000
50%,7.500000
75%,35.500000
max,63.000000


In [5]:
# Test scipy.stats.mstats mquantiles function on its own
# Incorporate into my five-, six-, and seven-number summary functions eventually.
# Need to test all options for alphap and betap. Following is R-8 (recommended).
# Finally, add code to do CI with method R-8.
resultd = ms.mquantiles(df['y'], prob=(0.25, 0.50, 0.75), alphap=0.33, betap=0.33)
resultd

array([ 0.415,  7.5  , 46.89 ])

In [6]:
resultd[0]

0.41500000000000004

In [7]:
resultminitab = ms.mquantiles(df['y'], prob=(0.25, 0.50, 0.75), alphap=0, betap=0)
resultminitab

array([ 0.25,  7.5 , 52.5 ])

In [8]:
ds.nonparametric_summary(df['y'])

,n,min,q1,q2,q3,iqr,max
interpolation,,,,,,,
linear,8,0,0.75,7.5,35.5,34.75,63
lower,8,0,0.00,2.0,27.0,27.00,63
higher,8,0,1.00,13.0,61.0,60.00,63
nearest,8,0,1.00,13.0,27.0,26.00,63
midpoint,8,0,0.50,7.5,44.0,43.50,63


In [9]:
# Calculate minimum value of the dataframe.
df.min()

y    0
dtype: int64

In [10]:
# Calculate maximum value of the dataframe.
df.max()

y    63
dtype: int64

In [11]:
# Calculate the average of the dataframe.
df.mean()

y    20.875
dtype: float64

In [12]:
# Calculate sample standard deviation of the dataframe.
df.std()

y    27.010249
dtype: float64

In [13]:
# Calculate sample variance of the dataframe.
df.var()

y    729.553571
dtype: float64

In [14]:
# Calculate skewness of the dataframe.
df.skew()

y    1.027767
dtype: float64

In [15]:
# Calculate kurtosis of the dataframe.
df.kurt()

y   -0.771495
dtype: float64

In [16]:
# Calculate the sample size of the dataframe.
df.count()

y    8
dtype: int64

In [17]:
adresult = anderson(df['y'], dist='norm')
adresult.statistic

0.8251508484815488

In [18]:
# Use statsmodels to calculate the Anderson-Darling test statistic and p value.
smd.normal_ad(df['y'])

/home/gilles/.pyenv/versions/3.6.6/lib/python3.6/site-packages/statsmodels/stats/_adnorm.py:66: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  S = np.sum((2*i[sl1]-1.0)/N*(np.log(z)+np.log(1-z[sl2])), axis=axis)


(0.8251508484815488, 0.018166132339072383)

In [19]:
# End of time estimation for the notebook.
end_time = dt.datetime.now()
(end_time - start_time).total_seconds()

1.014344

# References

[datasense module](https://github.com/gillespilon/datasense).

[Five-number summary](https://en.wikipedia.org/wiki/Five-number_summary).

Hyndman, Rob J. and Yanan Fan. "Sample Quantiles in Statistical Packages." *The American Statistician* Vol. 50, No. 4 (Nov. 1996): 361-365. [JSTOR 2684934](http://www.jstor.org/stable/2684934).

McGill, Robert, John W. Tukey, and Wayne A. Larsen. 1978. "Variations of Box Plots." *The American Statistician 21 (February 1978), no. 1; 12-16. [https://www.jstor.org/stable/2683468](https://www.jstor.org/stable/2683468).

[pandas](https://pandas.pydata.org/pandas-docs/stable/api.html).

[R documentation on quantiles](https://www.rdocumentation.org/packages/stats/topics/quantile).

[R quantile function](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/quantile.html).

[SciPy documentation on mquantiles](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.mquantiles.html).

[SciPy documentation on masked arrays](https://docs.scipy.org/doc/numpy-1.15.1/reference/maskedarray.html).

Wikipedia. "Quantile". Last modified 2018-08-01. [https://en.wikipedia.org/wiki/Quantile](https://en.wikipedia.org/wiki/Quantile).